In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key


In [2]:
# Read the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv( "weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(last 3hrs),Snow inches(last 3hrs),Current Description
0,0,Mahebourg,MU,2020-01-05 11:19:04,-20.41,57.70,81.00,83,75,8.05,0.00,0.0,broken clouds
1,1,Hobart,AU,2020-01-05 11:18:10,-42.88,147.33,60.80,55,20,11.41,0.00,0.0,few clouds
2,2,Yellowknife,CA,2020-01-05 11:15:11,62.45,-114.38,-2.00,84,90,13.87,0.00,0.0,light snow
3,3,Avarua,CK,2020-01-05 11:19:05,-21.21,-159.78,77.00,94,100,3.36,0.00,0.0,overcast clouds
4,4,Turukhansk,RU,2020-01-05 11:19:06,65.80,87.96,18.19,94,100,16.42,1.44,0.0,light snow


In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Ask the customer to preference raining/snowing or not? 
raining = str(input("Do you want it to be raining? (yes/no)"))
snowing = str(input("Do you want it to be snowing? (yes/no)"))


What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 100
Do you want it to be raining? (yes/no)yes
Do you want it to be snowing? (yes/no)yes


In [34]:
# Filter the dataset to find the cities that fit the criteria.
zero = 0.0
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       ( 
                                           ( (raining == 'yes') & (city_data_df["Rain inches(last 3hrs)"] > 0.0)) |
                                           ((snowing == 'yes' ) & (city_data_df["Snow inches(last 3hrs)"] > 0.0)) |
                                           
                                           ((raining == 'no') & (city_data_df["Rain inches(last 3hrs)"] == 0.0))  |
                                           ((snowing == 'no') & (city_data_df["Snow inches(last 3hrs)"] == 0.0))  
                                       )
                                      ]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(last 3hrs),Snow inches(last 3hrs),Current Description
4,4,Turukhansk,RU,2020-01-05 11:19:06,65.80,87.96,18.19,94,100,16.42,1.44,0.00,light snow
7,7,Mataura,NZ,2020-01-05 11:19:06,-46.19,168.86,50.00,77,64,11.01,0.00,0.31,light rain
8,8,Kavieng,PG,2020-01-05 11:19:07,-2.57,150.80,84.88,73,65,11.10,0.00,3.50,moderate rain
22,22,Kaliua,TZ,2020-01-05 11:19:11,-5.06,31.79,84.24,57,90,1.68,0.00,3.12,moderate rain
28,28,Atuona,PF,2020-01-05 11:19:12,-9.80,-139.03,79.92,71,3,5.37,0.00,0.31,light rain
37,37,Hithadhoo,MV,2020-01-05 11:19:14,-0.60,73.08,84.79,69,67,8.77,0.00,2.94,light rain
66,66,Salinopolis,BR,2020-01-05 11:19:22,-0.61,-47.36,79.86,79,100,5.97,0.00,6.44,moderate rain
67,67,Sambava,MG,2020-01-05 11:19:23,-14.27,50.17,79.14,94,100,1.81,0.00,7.50,moderate rain
78,78,Champerico,MX,2020-01-05 11:19:25,16.38,-93.60,59.36,87,99,5.28,0.00,0.13,light rain
79,79,Jamestown,AU,2020-01-05 11:19:26,-33.21,138.60,54.57,88,100,11.52,0.00,0.88,light rain


In [35]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [37]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
4,Turukhansk,RU,18.19,65.80,87.96,light snow,
7,Mataura,NZ,50.00,-46.19,168.86,light rain,
8,Kavieng,PG,84.88,-2.57,150.80,moderate rain,
22,Kaliua,TZ,84.24,-5.06,31.79,moderate rain,
28,Atuona,PF,79.92,-9.80,-139.03,light rain,
37,Hithadhoo,MV,84.79,-0.60,73.08,light rain,
66,Salinopolis,BR,79.86,-0.61,-47.36,moderate rain,
67,Sambava,MG,79.14,-14.27,50.17,moderate rain,
78,Champerico,MX,59.36,16.38,-93.60,light rain,
79,Jamestown,AU,54.57,-33.21,138.60,light rain,


In [38]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()  
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [44]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [45]:
# WeatherPy_vacation.csv 
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")  
#
